<a href="https://colab.research.google.com/github/hmatny/question_answering/blob/master/Anserini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Environment Setup

#### Download and install maven

In [20]:
!wget http://mirror.reverse.net/pub/apache/maven/maven-3/3.6.1/binaries/apache-maven-3.6.1-bin.tar.gz
!tar -xvzf apache-maven-3.6.1-bin.tar.gz -C /opt

--2019-04-15 21:55:58--  http://mirror.reverse.net/pub/apache/maven/maven-3/3.6.1/binaries/apache-maven-3.6.1-bin.tar.gz
Resolving mirror.reverse.net (mirror.reverse.net)... 208.100.14.200
Connecting to mirror.reverse.net (mirror.reverse.net)|208.100.14.200|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9136463 (8.7M) [application/x-gzip]
Saving to: ‘apache-maven-3.6.1-bin.tar.gz’

apache-maven-3.6.1- 100%[===================>]   8.71M  4.87MB/s    in 1.8s    

2019-04-15 21:56:00 (4.87 MB/s) - ‘apache-maven-3.6.1-bin.tar.gz’ saved [9136463/9136463]

apache-maven-3.6.1/README.txt
apache-maven-3.6.1/LICENSE
apache-maven-3.6.1/NOTICE
apache-maven-3.6.1/lib/
apache-maven-3.6.1/lib/slf4j-api.license
apache-maven-3.6.1/lib/checker-compat-qual.license
apache-maven-3.6.1/lib/jsr250-api.license
apache-maven-3.6.1/lib/jcl-over-slf4j.license
apache-maven-3.6.1/lib/org.eclipse.sisu.plexus.license
apache-maven-3.6.1/lib/animal-sniffer-annotations.license
apache-maven-3.6.

#### Set environment variables

In [7]:
import os
os.environ['PATH'] += ':/usr/lib/jvm/java-8-openjdk-amd64/bin:/opt/apache-maven-3.6.1/bin'
os.environ['JAVA_HOME'] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ['M2_HOME'] = "/opt/apache-maven-3.6.0"

# sanity check
!mvn -version

Apache Maven 3.6.1 (d66c9c0b3152b2e69ee9bac180bb8fcc8e6af555; 2019-04-04T19:00:29Z)
Maven home: /opt/apache-maven-3.6.1
Java version: 1.8.0_191, vendor: Oracle Corporation, runtime: /usr/lib/jvm/java-8-openjdk-amd64/jre
Default locale: en_US, platform encoding: UTF-8
OS name: "linux", version: "4.14.79+", arch: "amd64", family: "unix"


#### Clone Anserini git repo

In [33]:
!git clone https://github.com/castorini/Anserini.git fi
!cd Anserini && mvn clean package appassembler:assemble

[INFO] Scanning for projects...
Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-compiler-plugin/3.7.0/maven-compiler-plugin-3.7.0.pom (11 kB at 19 kB/s)
Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-plugins/30/maven-plugins-30.pom (10 kB at 308 kB/s)
Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-parent/30/maven-parent-30.pom (41 kB at 780 kB/s)
Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/apache/18/apache-18.pom (16 kB at 423 kB/s)
Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-compiler-plugin/3.7.0/maven-compiler-plugin-3.7.0.jar (58 kB at 1.4 MB/s)
Downloaded from central: https://repo.maven.apache.org/maven2/org/codehaus/mojo/appassembler-maven-plugin/2.0.0/appassembler-maven-plugin-2.0.0.pom (18 kB at 410 kB/s)
Downloaded from central: https://repo.maven.apache.org/maven2/org/code

#### Download the Lucene index

In [15]:
# Download the lucene index
!gsutil cp gs://cs378_bert/lucene-index.tar.gz /tmp/lucene-index.tar.gz
!cd /tmp && tar -xvzf lucene-index.tar.gz -C /content/Anserini/

Copying gs://cs378_bert/lucene-index.tar.gz...
/ [1 files][  6.6 MiB/  6.6 MiB]                                                
Operation completed over 1 objects/6.6 MiB.                                      
lucene-index.car17.pos+docvectors+rawdocs/
lucene-index.car17.pos+docvectors+rawdocs/_0.si
lucene-index.car17.pos+docvectors+rawdocs/_0.fnm
lucene-index.car17.pos+docvectors+rawdocs/_0.fdx
lucene-index.car17.pos+docvectors+rawdocs/_1.si
lucene-index.car17.pos+docvectors+rawdocs/_1.fdx
lucene-index.car17.pos+docvectors+rawdocs/_1.fnm
lucene-index.car17.pos+docvectors+rawdocs/_0.tvx
lucene-index.car17.pos+docvectors+rawdocs/segments_3
lucene-index.car17.pos+docvectors+rawdocs/_1.tvd
lucene-index.car17.pos+docvectors+rawdocs/_0.tvd
lucene-index.car17.pos+docvectors+rawdocs/_1.tvx
lucene-index.car17.pos+docvectors+rawdocs/_0_Lucene70_0.dvd
lucene-index.car17.pos+docvectors+rawdocs/_0_Lucene70_0.dvm
lucene-index.car17.pos+docvectors+rawdocs/_1_Lucene70_0.dvd
lucene-index.car17.pos+doc

#### Install dependencies

In [35]:
#!pip3 install pyjnius

  Stored in directory: /root/.cache/pip/wheels/c1/2d/85/9884050da2f10b9f72b029f34bedef0993c339437aa956906f
Successfully built pyjnius


### Using Anserini 

In [0]:
# Only run this once
import jnius_config
jnius_config.set_classpath('Anserini/target/anserini-0.4.1-SNAPSHOT-fatjar.jar')

In [14]:
from jnius import autoclass
JString = autoclass('java.lang.String')
JSearcher = autoclass('io.anserini.search.SimpleSearcher')

searcher = JSearcher(JString('Anserini/lucene-index.car17.pos+docvectors+rawdocs'))
hits = searcher.search(JString('hubble space telescope'))

for ind, hit in enumerate(hits):
        print('hit #' + str(ind))
        print('docid: ' + str(hit.docid))
        print('score: ' + str(hit.score))
        print('content: ')
        print(hit.content)
        print()

hit #0
docid: cc71d40f26b1c241314b35a0ca138497e3014d1f
score: 9.588899612426758
content: 
Instruments used by SETI Institute scientists include the ground-based Allen Telescope Array, several ground-based optical telescopes such as the Shane telescope at Lick Observatory, the W.M. Keck telescopes & IRTF in Hawaii, the Very Large Telescopes in Chile. SETI researchers also use space telescope facilities, mostly  the Hubble Space Telescope, the Spitzer Space Telescope and the Herschel Space Telescope. SETI scientists are also involved in space missions, the New Horizons mission toward Pluto, the Cassini mission, currently in orbit around Saturn, and the Mars Rovers Opportunity and Curiosity.

hit #1
docid: cc71d40f26b1c241314b35a0ca138497e3014d1f
score: 9.588898658752441
content: 
Instruments used by SETI Institute scientists include the ground-based Allen Telescope Array, several ground-based optical telescopes such as the Shane telescope at Lick Observatory, the W.M. Keck telescopes & I